In [43]:
# EDA
import pandas as pd
import numpy as np

# Data Preprocessing
from sklearn import preprocessing

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Recommender System Imps
# Content Based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative Based Filtering
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# To work with text data
import re
import string

In [44]:
df = pd.read_csv('foods.csv')
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [45]:
# No of dishes in my dataset
len(list(df['Name'].unique()))

400

In [46]:
df['C_Type'].unique() # Categorical Data

array(['Healthy Food', 'Snack', 'Dessert', 'Japanese', 'Indian', 'French',
       'Mexican', 'Italian', 'Chinese', 'Beverage', 'Thai', 'Korean',
       ' Korean', 'Vietnames', 'Nepalese', 'Spanish'], dtype=object)

In [47]:
df['Veg_Non'].unique() # Categorical Data

array(['veg', 'non-veg'], dtype=object)

In [48]:
len(df)

400

In [49]:
df.shape

(400, 5)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food_ID   400 non-null    int64 
 1   Name      400 non-null    object
 2   C_Type    400 non-null    object
 3   Veg_Non   400 non-null    object
 4   Describe  400 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.8+ KB


In [51]:
# Let's make a function to remove all the punctuation from the "Describe" column
def text_cleaning(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

In [52]:
# Let's clean the text
df['Describe'] = df['Describe'].apply(text_cleaning)

In [53]:
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,white balsamic vinegar lemon juice lemon rind ...
1,2,chicken minced salad,Healthy Food,non-veg,olive oil chicken mince garlic minced onion sa...
2,3,sweet chilli almonds,Snack,veg,almonds whole egg white curry leaves salt suga...
3,4,tricolour salad,Healthy Food,veg,vinegar honeysugar soy sauce salt garlic clove...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...


In [54]:
# Are there any duplicate data ?
df.duplicated().sum()
# None

0

In [55]:
df.isnull().sum()

Food_ID     0
Name        0
C_Type      0
Veg_Non     0
Describe    0
dtype: int64

In [56]:
# General Description
df.describe()

,Food_ID
count,400.000000
mean,200.500000
std,115.614301
min,1.000000
25%,100.750000
50%,200.500000
75%,300.250000
max,400.000000


**CONTENT BASED FILTERING**

In [57]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Describe'])
tfidf_matrix.shape

(400, 1261)

In [58]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.16228366, 0.13001124, ..., 0.1286286 , 0.04277223,
        0.09993639],
       [0.16228366, 1.        , 0.06799336, ..., 0.14878001, 0.05688681,
        0.16917639],
       [0.13001124, 0.06799336, 1.        , ..., 0.03291577, 0.11795401,
        0.01834168],
       ...,
       [0.1286286 , 0.14878001, 0.03291577, ..., 1.        , 0.        ,
        0.10087579],
       [0.04277223, 0.05688681, 0.11795401, ..., 0.        , 1.        ,
        0.        ],
       [0.09993639, 0.16917639, 0.01834168, ..., 0.10087579, 0.        ,
        1.        ]])

In [59]:
# Just considering the Food names from the dataframe
indices = pd.Series(df.index, index=df['Name']).drop_duplicates()
indices

Name
summer squash salad                                          0
chicken minced salad                                         1
sweet chilli almonds                                         2
tricolour salad                                              3
christmas cake                                               4
                                                          ... 
Kimchi Toast                                               395
Tacos de Gobernador (Shrimp, Poblano, and Cheese Tacos)    396
Melted Broccoli Pasta With Capers and Anchovies            397
Lemon-Ginger Cake with Pistachios                          398
Rosemary Roasted Vegetables                                399
Length: 400, dtype: int64

In [60]:
# The main recommender code!
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar food
    sim_scores = sim_scores[1:6]

    food_indices = [i[0] for i in sim_scores]
    return df['Name'].iloc[food_indices]

**Advanced Content Based Filtering**

In [61]:
# Including all features that will help in recommending better
features = ['C_Type','Veg_Non', 'Describe']

In [62]:
# Soup represents a mixture of elements
# Similarly, I am making one column that will have all the important features
# I am simply concatenating the strings

def create_soup(x):
    return x['C_Type'] + " " + x['Veg_Non'] + " " + x['Describe']

In [63]:
# Using the soup(), I am creating the column for the dataframe df
df['soup'] = df.apply(create_soup, axis=1)

In [64]:
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe,soup
0,1,summer squash salad,Healthy Food,veg,white balsamic vinegar lemon juice lemon rind ...,Healthy Food veg white balsamic vinegar lemon ...
1,2,chicken minced salad,Healthy Food,non-veg,olive oil chicken mince garlic minced onion sa...,Healthy Food non-veg olive oil chicken mince g...
2,3,sweet chilli almonds,Snack,veg,almonds whole egg white curry leaves salt suga...,Snack veg almonds whole egg white curry leaves...
3,4,tricolour salad,Healthy Food,veg,vinegar honeysugar soy sauce salt garlic clove...,Healthy Food veg vinegar honeysugar soy sauce ...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...,Dessert veg christmas dry fruits presoaked ora...


In [65]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [66]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [67]:
# Reseting the index and pulling out the names of the food alone from the df dataframe
df = df.reset_index()
indices = pd.Series(df.index, index=df['Name'])

In [68]:
# Let's see the names of the food pulled out
display(indices)

Name
summer squash salad                                          0
chicken minced salad                                         1
sweet chilli almonds                                         2
tricolour salad                                              3
christmas cake                                               4
                                                          ... 
Kimchi Toast                                               395
Tacos de Gobernador (Shrimp, Poblano, and Cheese Tacos)    396
Melted Broccoli Pasta With Capers and Anchovies            397
Lemon-Ginger Cake with Pistachios                          398
Rosemary Roasted Vegetables                                399
Length: 400, dtype: int64

In [69]:
# This is the first model - simple variation
get_recommendations('tricolour salad')

103             chilli chicken
1         chicken minced salad
27     vegetable som tam salad
282          veg hakka noodles
166             veg fried rice
Name: Name, dtype: object

In [70]:
# This is the second model - advanced variation
get_recommendations('tricolour salad', cosine_sim2)

1                         chicken minced salad
103                             chilli chicken
27                     vegetable som tam salad
177                        oats shallots pulao
69     shepherds salad (tamatar-kheera salaad)
Name: Name, dtype: object

**COLLABORATIVE FILTERING**

In [71]:
# Importing the ratings file
rating = pd.read_csv('ratings.csv')
rating.head()

,User_ID,Food_ID,Rating
0,1.0,88.0,4.0
1,1.0,46.0,3.0
2,1.0,24.0,5.0
3,1.0,25.0,4.0
4,2.0,49.0,1.0


In [72]:
rating.isnull().sum()

User_ID    1
Food_ID    1
Rating     1
dtype: int64

In [73]:
# I actually saw the data earlier and found that the last row had no values
# Let's see
rating.tail()

,User_ID,Food_ID,Rating
507,99.0,22.0,1.0
508,100.0,24.0,10.0
509,100.0,233.0,10.0
510,100.0,29.0,7.0
511,NaN,NaN,NaN


In [74]:
# Removing the last row
rating = rating[:511]
rating.tail()

,User_ID,Food_ID,Rating
506,99.0,65.0,7.0
507,99.0,22.0,1.0
508,100.0,24.0,10.0
509,100.0,233.0,10.0
510,100.0,29.0,7.0


In [75]:
rating.isnull().sum()

User_ID    0
Food_ID    0
Rating     0
dtype: int64

In [76]:
# Making a dataframe that has food ID and the number of ratings
food_rating = rating.groupby(by = 'Food_ID').count()
food_rating = food_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
food_rating

,Food_ID,Rating_count
0,1.0,2
1,2.0,3
2,3.0,2
3,4.0,2
4,5.0,6
...,...,...
304,305.0,1
305,306.0,1
306,307.0,1
307,308.0,1


In [77]:
# Food rating dataframe description
food_rating['Rating_count'].describe()

count    309.000000
mean       1.653722
std        1.107748
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        7.000000
Name: Rating_count, dtype: float64

In [78]:
# The user rating dataframe shows the number of ratings given with respect to the user
user_rating = rating.groupby(by='User_ID').count()
user_rating = user_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating

,User_ID,Rating_count
0,1.0,4
1,2.0,4
2,3.0,9
3,4.0,6
4,5.0,6
...,...,...
95,96.0,6
96,97.0,7
97,98.0,7
98,99.0,6


In [79]:
# User rating dataframe description
user_rating["Rating_count"].describe()

count    100.000000
mean       5.110000
std        2.352282
min        1.000000
25%        3.000000
50%        5.000000
75%        7.000000
max       11.000000
Name: Rating_count, dtype: float64

In [80]:
# Ultimate Table
rating_matrix = rating.pivot_table(index='Food_ID',columns='User_ID',values='Rating').fillna(0)
rating_matrix.head()

User_ID,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
Food_ID,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0


In [81]:
# Shape of rating_matrix
rating_matrix.shape

(309, 100)

In [82]:
# What is csr_matrix?
# Sparse matrix is a matrix that contains a lot of zeros. This functions helps to compress the sparse matrix.
# Thanks to https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

csr_rating_matrix =  csr_matrix(rating_matrix.values)
print(csr_rating_matrix)

  (0, 48)	5.0
  (0, 70)	10.0
  (1, 8)	3.0
  (1, 21)	5.0
  (1, 38)	10.0
  (2, 76)	1.0
  (2, 88)	7.0
  (3, 40)	6.0
  (3, 89)	6.0
  (4, 27)	10.0
  (4, 38)	10.0
  (4, 72)	7.0
  (4, 87)	3.0
  (4, 93)	2.0
  (4, 97)	7.0
  (5, 42)	10.0
  (5, 48)	6.0
  (5, 76)	6.0
  (5, 91)	1.0
  (6, 16)	4.0
  (6, 47)	5.0
  (6, 62)	9.0
  (6, 69)	8.0
  (6, 70)	8.0
  (7, 4)	6.0
  :	:
  (284, 30)	9.0
  (285, 80)	6.0
  (286, 24)	3.0
  (287, 54)	3.0
  (288, 55)	9.0
  (289, 31)	7.0
  (290, 15)	1.0
  (291, 2)	8.0
  (292, 95)	5.0
  (293, 41)	4.0
  (294, 43)	10.0
  (295, 41)	10.0
  (296, 94)	5.0
  (297, 55)	4.0
  (298, 2)	1.0
  (299, 28)	9.0
  (300, 53)	1.0
  (301, 77)	5.0
  (302, 63)	6.0
  (303, 29)	1.0
  (304, 55)	9.0
  (305, 79)	8.0
  (306, 70)	1.0
  (307, 96)	3.0
  (308, 31)	5.0


In [83]:
# Using cosine similarity to find nearest neigbours
recommender = NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)

NearestNeighbors(metric='cosine')

In [84]:
# The main recommender code!
def Get_Recommendations(title):
    user= df[df['Name']==title]
    user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]
    user_ratings = rating_matrix.iloc[user_index]

    reshaped = user_ratings.values.reshape(1,-1)
    distances, indices = recommender.kneighbors(reshaped,n_neighbors=16)

    nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors = pd.DataFrame({'Food_ID': nearest_neighbors_indices})

    result = pd.merge(nearest_neighbors,df,on='Food_ID',how='left')

    return result.head()

In [85]:
# Get recommendations with this function
Get_Recommendations('tricolour salad')

C:\Users\Asus\AppData\Local\Temp\ipykernel_9080\4088314846.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]


,Food_ID,index,Name,C_Type,Veg_Non,Describe,soup
0,126.0,125,andhra crab meat masala,Indian,non-veg,processed crab meat refined oil curry leaves g...,Indian non-veg processed crab meat refined oil...
1,75.0,74,detox haldi tea,Beverage,veg,haldi ginger black pepper honey water,Beverage veg haldi ginger black pepper honey w...
2,100.0,99,spicy chicken curry,Indian,non-veg,oil ghee onion paste garlic paste ginger paste...,Indian non-veg oil ghee onion paste garlic pas...
3,259.0,258,ragi coconut ladoo (laddu),Dessert,veg,finger millet flour ragi jaggery peanuts cocon...,Dessert veg finger millet flour ragi jaggery p...
4,51.0,50,christmas chocolate fudge cookies,Dessert,veg,unsalted butter brown sugar chocolate chocolat...,Dessert veg unsalted butter brown sugar chocol...


In [86]:
Get_Recommendations('christmas cake')

C:\Users\Asus\AppData\Local\Temp\ipykernel_9080\4088314846.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]


,Food_ID,index,Name,C_Type,Veg_Non,Describe,soup
0,50.0,49,french pork chop,French,veg,pork chop pink pepper corn green pepper corn a...,French veg pork chop pink pepper corn green pe...
1,162.0,161,prawn potato soup,Thai,veg,Onion potato prawns eggs milk butter coriander...,Thai veg Onion potato prawns eggs milk butter ...
2,256.0,255,apple kheer,Dessert,veg,apples basmati rice nuscovado sugar you can al...,Dessert veg apples basmati rice nuscovado suga...
3,195.0,194,egg in a blanket,French,non-veg,eggs brown bread slices butter chilli flakes o...,French non-veg eggs brown bread slices butter ...
4,30.0,29,couscous with ratatouille - tangy tomato sauce,French,veg,for the cous cous plain couscous extra virgin ...,French veg for the cous cous plain couscous ex...


In [87]:
df.head(10)

,index,Food_ID,Name,C_Type,Veg_Non,Describe,soup
0,0,1,summer squash salad,Healthy Food,veg,white balsamic vinegar lemon juice lemon rind ...,Healthy Food veg white balsamic vinegar lemon ...
1,1,2,chicken minced salad,Healthy Food,non-veg,olive oil chicken mince garlic minced onion sa...,Healthy Food non-veg olive oil chicken mince g...
2,2,3,sweet chilli almonds,Snack,veg,almonds whole egg white curry leaves salt suga...,Snack veg almonds whole egg white curry leaves...
3,3,4,tricolour salad,Healthy Food,veg,vinegar honeysugar soy sauce salt garlic clove...,Healthy Food veg vinegar honeysugar soy sauce ...
4,4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...,Dessert veg christmas dry fruits presoaked ora...
5,5,6,japanese curry arancini with barley salsa,Japanese,veg,japanese curry sticky rice cheese inside rice ...,Japanese veg japanese curry sticky rice cheese...
6,6,7,chocolate nero cookies,Dessert,veg,almonds eggs granulated sugar bittersweet choc...,Dessert veg almonds eggs granulated sugar bitt...
7,7,8,lamb and chargrilled bell pepper soup,Healthy Food,non-veg,lamb bones preferably shank and shoulder onion...,Healthy Food non-veg lamb bones preferably sha...
8,8,9,cream of almond soup,Healthy Food,veg,vegetable stock skimmed milk toasted almonds p...,Healthy Food veg vegetable stock skimmed milk ...
9,9,10,broccoli and almond soup,Healthy Food,veg,vegetable stock broccoli ground almonds toaste...,Healthy Food veg vegetable stock broccoli grou...


In [88]:
Get_Recommendations("broccoli and almond soup")

C:\Users\Asus\AppData\Local\Temp\ipykernel_9080\4088314846.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]


,Food_ID,index,Name,C_Type,Veg_Non,Describe,soup
0,208.0,207,fennel scented sweet banana fritters,Dessert,veg,Wheat flour banana jaggery milk or water ghee ...,Dessert veg Wheat flour banana jaggery milk or...
1,210.0,209,quinoa coconut crumble custard,Dessert,veg,Knoia cooked oats cinnamon powder salt brown s...,Dessert veg Knoia cooked oats cinnamon powder ...
2,181.0,180,vegetable biryani,Indian,veg,Cumin Onion Ginger Garlic Mix Vegetable Corian...,Indian veg Cumin Onion Ginger Garlic Mix Veget...
3,48.0,47,almond and chicken momos (without shell),Chinese,non-veg,chicken mince garlic carrots spring onion ging...,Chinese non-veg chicken mince garlic carrots s...
4,291.0,290,chicken razala,Indian,non-veg,chicken cooked in a rich gravy with mint,Indian non-veg chicken cooked in a rich gravy ...


In [89]:
import pickle

In [92]:
from sklearn.pipeline import Pipeline

In [94]:
pickle.dump(df, open('df.pkl', 'wb'))